In [2]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [3]:
PROBLEM_SIZE = 100
NUM_SETS = 90
#returns a random number between 0 and 1. If the number is less than 0.3, the corresponding element in the array will be True, otherwise it will be False. 
# This code could be used to generate a data set for the set covering problem or a similar problem.
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])


In [4]:
def goal_check(state):
     #The function checks whether all elements of a set are covered by the selected subsets. 
     #Uses the reduce function with the np.logical_or operator to combine all the selected subsets into a single Boolean array. 
     #Then, use the np.all function to check whether all elements of the set are covered. 
     #If all elements are covered, the function returns True, otherwise it returns False.
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [5]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [35]:
def path_cost(state):

    return sum(reduce(np.logical_or, [SETS[i] for i in state.taken],np.array([False for _ in range(PROBLEM_SIZE)])))

# another possible cost function would be the number of taken tiles: the smaller number of taken ones the better
def g(state):
    return len(state.taken)

#if we want to apply a greedy approach we have to define the concept of distance between the actual state and the goal solution: like, how much taken tiles I have to add to find a goal state?
def distance(state):
    return PROBLEM_SIZE - sum(reduce(np.logical_or, [SETS[i] for i in state.taken],np.array([False for _ in range(PROBLEM_SIZE)])))

def is_special(i):
    return sum(SETS[i] == True)  >= ((PROBLEM_SIZE / 2) + PROBLEM_SIZE/10)

#################   RIVEDI STA PARTE QUA!!!!   ############

def calculate_order_factor(state):
    # Ordina gli insiemi in state.not_taken in base al criterio desiderato
    ordered_sets = sorted(state.not_taken, key=lambda i: sum(SETS[i]), reverse=True)
    
    # Crea un nuovo stato con gli insiemi riordinati
    new_state = State(state.taken, ordered_sets)
    
    return new_state

##########################################################


def h(state):
    #order according to mumber of sets coverable by the not taken sets 
    state = calculate_order_factor(state)

    # compute the distance from the goal state
    dist = distance(state)
    
    # consider the special sets - see the is_special function to see what kind of sets we considers as special
    special_sets = [i for i in state.not_taken if is_special(i)]
    num_special_sets = len(special_sets)

    return  dist - num_special_sets




In [28]:
def a_f(state):
    return g(state) + h(state)

In [36]:

####  A STAR APPROACH  ####

frontier = PriorityQueue() # we use a PriorityQueue because we want to define a cost funzion a_f = g(state) + h(state)
initial_state = State(set(), set(range(NUM_SETS)))
frontier.put((a_f(initial_state),initial_state)) #the first state is the one with no taken sets

counter = 0 #counter used just to count the number of occurrencies needed to solve the problem
_,current_state = frontier.get()  #start the resolution taking the first element from the frontier queue
while not goal_check(current_state):    #iterate until the problem is not resolved
    counter += 1
    for action in current_state[1]: #an ACTION is represented as the activity of taking one set from 
        # The ^ operator in Python is a bitwise XOR (exclusive OR) operator. It returns True if and only if its arguments differ (one is True, the other is False)
        #so here it equals to take an action (set) from not_taken and put it into taken
        # new_state = State(current_state.taken | {action}, current_state.not_taken - {action}) -> this would be the same
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})

        #it puts all the states generated into the frontier queue
        frontier.put((a_f(new_state),new_state))
    
    #endly it takes one state at time and analyze its condition (if can be considered a goal state in the while above there)
    _,current_state = frontier.get()

print(f"Solved in {counter:,} steps")
print(a_f(current_state))
current_state

Solved in 6 steps
6


State(taken={67, 36, 8, 81, 22, 29}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89})

In [9]:
current_state

State(taken={67, 36, 8, 81, 22, 29}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89})

In [10]:
goal_check(current_state)

True